In [19]:
! git clone https://github.com/cdli-gh/data

Cloning into 'data'...
remote: Enumerating objects: 7150, done.
remote: Counting objects: 100% (464/464), done.
remote: Compressing objects: 100% (335/335), done.
remote: Total 7150 (delta 136), reused 452 (delta 129), pack-reused 6686
Receiving objects: 100% (7150/7150), 5.90 GiB | 22.07 MiB/s, done.
Resolving deltas: 100% (4104/4104), done.


In [11]:
! cd data; git lfs status

On branch master
Objects to be pushed to origin/master:


Objects to be committed:


Objects not staged for commit:




In [12]:
import pandas as pd

In [13]:
COLS = [
#     'ark_number',
    'id_text',
#     'id_text2',
#     'author',
    'genre',
    'subgenre',
    'period',
#     'composite_id',
#     'external_id',
    'id',
    'photo_up'
]

['id', 'genre', 'period', 'subgenre', 'photo_up', 'id_text']

['id', 'genre', 'period', 'subgenre', 'photo_up', 'id_text']

In [18]:
df = pd.read_csv("https://media.githubusercontent.com/media/cdli-gh/data/master/cdli_cat.csv", usecols=COLS)

/storage/modules/packages/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (46) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
df.shape

(353283, 6)

In [20]:
df.sample(5)

,genre,id,id_text,period,photo_up,subgenre
82032,Royal/Monumental,82033,233985,Lagash II (ca. 2200-2100 BC),NaN,witness
52007,Administrative,52008,144224,Ur III (ca. 2100-2000 BC),NaN,NaN
320746,Letter,329556,498979,Old Assyrian (ca. 1950-1850 BC),NaN,NaN
216697,Administrative,216744,381273,Ur III (ca. 2100-2000 BC),NaN,NaN
100664,Literary,100666,250888,Old Babylonian (ca. 1900-1600 BC),600ppi 20160630,ETCSL 2.04.02.01 Šulgi A ('Decad no. 01') (wit...


In [14]:
df = pd.read_csv('data/cdli_cat.csv', keep_default_na=False, usecols=COLS)

ValueError: Usecols do not match columns, columns expected but not found: ['id', 'photo_up', 'period', 'genre', 'subgenre', 'id_text']

In [21]:
df.shape

(353283, 6)

In [37]:
df = df[df.photo_up.notna()].copy()

In [38]:
df.shape

(103498, 8)

In [39]:
df.head()

,genre,id,id_text,period,photo_up,subgenre,period_normed,era
660,Lexical,661,713,Uruk III (ca. 3200-3000 BC),600ppi 20160630,Archaic Vessels and Garments (witness),Uruk III,early_bronze
661,Lexical,662,714,Uruk III (ca. 3200-3000 BC),600ppi 20160630,Archaic Geography (witness),Uruk III,early_bronze
662,Lexical,663,723,ED I-II (ca. 2900-2700 BC),600ppi 20160630,ED Lu2 A (witness),ED I-II,early_bronze
663,Lexical,664,724,ED I-II (ca. 2900-2700 BC),600ppi 20160913,Archaic Fish (witness),ED I-II,early_bronze
667,Lexical,668,728,ED I-II (ca. 2900-2700 BC),600ppi 20160630,ED Lu2 A (witness),ED I-II,early_bronze


In [40]:
df['period_normed'] = (
    df.period.str.replace(
        r'\?', '').str.replace(
        r'\(.*\)', '').str.replace(
        ' +', ' ').str.strip()
)

/storage/modules/packages/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/storage/modules/packages/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/storage/modules/packages/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [26]:
df = df[df.period_normed != ''].copy()

In [27]:
df.shape

(353283, 7)

In [41]:
df.sample(5)

,genre,id,id_text,period,photo_up,subgenre,period_normed,era
12709,Administrative,12710,103813,Ur III (ca. 2100-2000 BC),600ppi 20160630,NaN,Ur III,early_bronze
69965,Administrative ?,69966,221224,ED IIIb (ca. 2500-2340 BC),600ppi 20160913,NaN,ED IIIb,early_bronze
130232,Scientific,130270,285267,NaN,600ppi 20160630,medical,NaN,
75610,Administrative,75611,227208,Ebla (ca. 2350-2250 BC),600ppi 20160630,NaN,Ebla,early_bronze
248323,Literary,248372,414073,Neo-Assyrian (ca. 911-612 BC),600ppi 20160913,NaN,Neo-Assyrian,iron


In [42]:
EARLY_BRONZE = {
    'Old Akkadian', 'Ur III',
    'ED IIIb', 'Uruk III',
    'Proto-Elamite', 'Lagash II',
    'Ebla', 'ED IIIa', 'ED I-II',
    'Uruk IV', 'Linear Elamite',
    'Harappan'
    
}
MID_LATE_BRONZE = {
    'Early Old Babylonian',
    'Old Babylonian', 'Old Assyrian',
    'Middle Babylonian', 'Middle Assyrian',
    'Middle Elamite', 'Middle Hittite'
}
IRON = {
    'Neo-Babylonian', 'Neo-Assyrian',
    'Achaemenid', 'Hellenistic',
    'Neo-Elamite'
}

In [30]:
ERA_MAP = {
    **{K: 'early_bronze' for K in EARLY_BRONZE},
    **{K: 'mid_late_bronze' for K in MID_LATE_BRONZE},
    **{K: 'iron' for K in IRON},
}

In [31]:
df['era'] = df.period_normed.map(ERA_MAP).fillna('')

In [32]:
df.sample(10)

,genre,id,id_text,period,photo_up,subgenre,period_normed,era
265166,NaN,265216,432814,NaN,NaN,NaN,NaN,
175943,Letter,175981,338779,Old Babylonian (ca. 1900-1600 BC),NaN,NaN,Old Babylonian,mid_late_bronze
30202,Administrative,30203,121363,Ur III (ca. 2100-2000 BC),600ppi 20160913,NaN,Ur III,early_bronze
118450,NaN,118452,270443,NaN,NaN,NaN,NaN,
203477,Administrative,203515,367485,Neo-Babylonian (ca. 626-539 BC),NaN,NaN,Neo-Babylonian,iron
344737,Prayer/Incantation,353572,523735,Old Babylonian (ca. 1900-1600 BC),NaN,NaN,Old Babylonian,mid_late_bronze
109788,Administrative,109790,260536,Neo-Babylonian (ca. 626-539 BC),600ppi 20160630,NaN,Neo-Babylonian,iron
185892,Administrative,185930,349041,Old Babylonian (ca. 1900-1600 BC),NaN,NaN,Old Babylonian,mid_late_bronze
299928,NaN,299985,468177,NaN,NaN,NaN,NaN,
250823,NaN,250872,416636,Middle Babylonian (ca. 1400-1100 BC),NaN,NaN,Middle Babylonian,mid_late_bronze


In [43]:
df.era.value_counts()

early_bronze       36634
iron               30800
mid_late_bronze    29689
                    6375
Name: era, dtype: int64

In [44]:
df.period_normed.nunique()

32

In [45]:
df.period_normed[df.era == ''].nunique()

8

In [46]:
df.period_normed[df.era == ''].value_counts()

uncertain               355
fake                     99
Pre-Uruk V               78
Uruk V                   46
Early Neo-Babylonian     11
Egyptian 0                2
Sassanian                 1
copy                      1
Name: period_normed, dtype: int64

In [50]:
df[df.period_normed != ''].to_csv('cdli_catalogue_data.csv', index=False)